# IMPLEMENTACIÓN DE PHI-3 EN GOOGLE CLOUD PLATFORM

In [1]:
# Origen
REGION = "us-central1"
PROJECT_ID = "vernal-guide-414621"
BIGQUERY = "jgj767"
TABLE = "igilñ"


In [ ]:
# Destino 
REGION = "us-central1"
PROJECT_ID = "vernal-guide-414621"
DATASET_ID = "news"  
TABLE_ID= "Graph"

## Definir la función que extraera las noticias filtradas, dejando en una tabla de BigQuery nueva solamente las noticias que involucren personas políticas y sean de temas políticos
### Función .py que ira a google functions y se automatizara semanalmente con google scheduler

Notas: Buscar las noticias con la mayor cantidad de personas, para hacer que la inferencia recoga la mayor cantidad de información, asi aprovechar los recursos y poder realizarlo economicamente. Ojala llegar a alrededor de un maximo 600-750-1000 noticias semanales para en un dia realizar una tanda de inferencia.
Maximizar personas y minimizar palabras. 


Proximo paso, generar una columna con el numero de palabras y el numero de personas, definir cuantas noticias uno quiere semanalmente ( apropiado = 750), generar la funcion de google cloud function, implementarla en google scheduler, implementar el modelo semanalmente (1 tanda de inferencia), arreglar manualmente la tabla. Ultimo paso, teniendo una basta cantidad de informacion, entrenar el modelo definitivo, implementarlo en un endpoint. analizar la red.

In [ ]:
from google.cloud import bigquery
import pandas as pd
import re
from google.cloud.exceptions import NotFound
import json
import datetime
personas_red_politica_dict = {'Maria Acevedo': 'diputado', 'Eric Aedo': 'diputado', 'Yovana Ahumada': 'diputado', 'Jorge Alessandri': 'diputado', 'Rene Alinco': 'diputado', 'Jaime Araya': 'diputado', 'Cristian Araya': 'diputado', 'Monica Arce': 'diputado', 'Roberto Arroyo': 'diputado', 'Danisa Astudillo': 'diputado', 'Chiara Barchiesi': 'diputado', 'Boris Barrera': 'diputado', 'Hector Barria': 'diputado', 'Miguel Becker': 'diputado', 'Maria Francisca Bello': 'diputado', 'Juan Carlos Beltran': 'diputado', 'Gustavo Benavente': 'diputado', 'Bernardo Berger': 'diputado', 'Alejandro Bernales': 'diputado', 'Carlos Bianchi': 'senador', 'Sergio Bobadilla': 'diputado', 'Fernando Borquez': 'diputado', 'Ana Maria Bravo': 'diputado', 'Marta Bravo': 'diputado', 'Jorge Brito': 'diputado', 'Felix Bugueno': 'diputado', 'Mercedes Bulnes': 'diputado', 'Miguel Angel Calisto': 'diputado', 'Felipe Camano': 'diputado', 'Karol Cariola': 'diputado', 'Alvaro Carter': 'diputado', 'Nathalie Castillo': 'diputado', 'Jose Miguel Castro': 'diputado', 'Andres Celis': 'diputado', 'Daniella Cicardini': 'diputado', 'Sofia Cid': 'diputado', 'Ricardo Cifuentes': 'diputado', 'Juan Antonio Coloma': 'diputado', 'Sara Concha': 'diputado', 'Maria Luisa Cordero': 'diputado', 'Eduardo Cornejo': 'diputado', 'Luis Cuello': 'diputado', 'Gonzalo De la Carrera': 'diputado', 'Tomas Derementeria': 'diputado', 'Catalina del Real': 'diputado', 'Viviana Delgado': 'diputado', 'Felipe Donoso': 'diputado', 'Jorge Duran': 'diputado', 'Eduardo Duran': 'diputado', 'Camila Flores': 'diputado', 'Lorena Fries': 'diputado', 'Juan Fuenzalida': 'diputado', 'Ana Maria Gazmuri': 'diputado', 'Andres Giordano': 'diputado', 'Felix Gonzalez': 'diputado', 'Marta Gonzalez': 'diputado', 'Mauro Gonzalez': 'diputado', 'Jorge Guzman': 'diputado', 'Carmen Hertz': 'diputado', 'Tomas Hirsch': 'diputado', 'Diego Ibanez': 'diputado', 'Marcos Ilabaca': 'diputado', 'Juan Irarrazaval': 'diputado', 'Pamela Jiles': 'diputado', 'Andres Jouannet': 'diputado', 'Harry Jurgensen': 'diputado', 'Johannes Kaiser': 'diputado', 'Cristian Labbe': 'diputado', 'Paula Labra': 'diputado', 'Tomas Lagomarsino': 'diputado', 'Joaquin Lavin': 'diputado', 'Henry Leal': 'diputado', 'Enrique Lee': 'diputado', 'Raul Leiva': 'diputado', 'Daniel Lilayu': 'diputado', 'Andres Longton': 'diputado', 'Luis Malla': 'diputado', 'Daniel Manouchehri': 'diputado', 'Cristobal Martinez': 'diputado', 'Carolina Marzan': 'diputado', 'Christian Matheson': 'diputado', 'Karen Medina': 'diputado', 'Cosme Mellado': 'diputado', 'Miguel Mellado': 'diputado', 'Daniel Melo': 'diputado', 'Jose Meza': 'diputado', 'Vlado Mirosevic': 'diputado', 'Claudia Mix': 'diputado', 'Helia Molina': 'diputado', 'Javiera Morales': 'diputado', 'Carla Morales': 'diputado', 'Cristhian Moreira': 'diputado', 'Benjamin Moreno': 'diputado', 'Jaime Mulet': 'diputado', 'Francesca Munoz': 'diputado', 'Camila Musante': 'diputado', 'Jaime Naranjo': 'diputado', 'Gloria Naveillan': 'diputado', 'Emilia Nuyado': 'diputado', 'Ericka Nanco': 'diputado', 'Mauricio Ojeda': 'diputado', 'Erika Olivera': 'diputado', 'Maite Orsini': 'diputado', 'Ximena Ossandon': 'diputado', 'Ruben Oyarzo': 'diputado', 'Hernan Palma': 'diputado', 'Marlene Perez': 'diputado', 'Joanna Perez': 'diputado', 'Catalina Perez': 'diputado', 'Victor Pino': 'diputado', 'Lorena Pizarro': 'diputado', 'Alejandra Placencia': 'diputado', 'Francisco Pulgar': 'diputado', 'Guillermo Ramirez': 'diputado', 'Matias Ramirez': 'diputado', 'Marcia Raphael': 'diputado', 'Jorge Rathgeb': 'diputado', 'Hugo Rey': 'diputado', 'Marcela Riquelme': 'diputado', 'Grivas': 'diputado', 'Camila Rojas': 'diputado', 'Agustin Romero': 'diputado', 'Leonidas Romero': 'diputado', 'Natalia Romero': 'diputado', 'Patricio Rosas': 'diputado', 'Jaime Saez': 'diputado', 'Jorge Saffirio': 'diputado', 'Clara Sagardia': 'diputado', 'Luis Sanchez': 'diputado', 'Juan Santana': 'diputado', 'Marisela Santibanez': 'diputado', 'Frank Sauerbaum': 'diputado', 'Diego Schalper': 'diputado', 'Emilia Schneider': 'diputado', 'Stephan Schubert': 'diputado', 'Alexis Sepulveda': 'diputado', 'Daniela Serrano': 'diputado', 'Leonardo Soto': 'diputado', 'Raul Soto': 'diputado', 'Marco Sulantay': 'diputado', 'Cristian Tapia': 'diputado', 'Hotuiti Teao': 'diputado', 'Carolina Tello': 'diputado', 'Renzo Trisot\xa0\xa0t\xa0\xa0i\xa0\xa0': 'diputado', 'Hector Ulloa': 'diputado', 'Francisco Undurraga': 'diputado', 'Alberto Undurraga': 'diputado', 'Cristobal Urruticoechea': 'diputado', 'Consuelo Veloso': 'diputado', 'Nelson Venegas': 'diputado', 'Sebastian Videla': 'diputado', 'Gaston Von Muhlenbrock': 'diputado', 'Flor Weisse': 'diputado', 'Gonzalo Winter': 'diputado', 'Gael Yeomans': 'diputado', 'José Miguel Insulza': 'senador', 'José Durana Semir': 'senador', 'Jorge Soria ': 'senador', 'Luz Ebensperger ': 'senador', 'Pedro Araya ': 'senador', 'Esteban Velásquez ': 'senador', 'Paulina Núñez ': 'senador', 'Yasna Provoste': 'senador', 'Rafael Prohens': 'senador', 'Daniel Núñez\xa0': 'senador', 'Matias Walker': 'senador', 'Sergio Gahona': 'senador', 'Fracisco Chahuan': 'senador', 'Kenneth Pugh': 'senador', 'Ricardo Lagos Weber': 'senador', 'Isabel Allende': 'senador', 'Juan Ignacio Latorre': 'senador', 'Manuel Jose Ossandon': 'senador', 'Luciano Cruz Coke ': 'senador', 'Claudia Pascual': 'senador', 'Fabiola Campillai': 'senador', 'Rojo Edwards': 'senador', 'Alejandro Sepulveda': 'senador', 'Javier Macaya': 'senador', 'Juan Luis Castro Gonzalez': 'senador', 'Juan Castro Prieto': 'senador', 'Rodrigo Galilea': 'senador', 'Juan Antonio Coloma Correa': 'senador', 'Ximena Rincon': 'senador', 'Paulina Vodanovic': 'senador', 'Sebastian Keitel Bianchi': 'senador', 'Enrique van Rysselberghe': 'senador', 'Gaston Saavedra': 'senador', 'Loreto Carvajal': 'senador', 'Gustavo Sanhueza': 'senador', 'Felipe Kast': 'senador', 'Francisco Huenchumilla': 'senador', 'Jaime Quintana': 'senador', 'Jose Garcia': 'senador', 'Carmen Gloria Aravena': 'senador', 'Alfonso Longton': 'senador', 'Maria Jose Gatica': 'senador', 'Ivan Flores': 'senador', 'Ivan Moreira ': 'senador', 'Fidel Espinoza': 'senador', 'Carlos Kuschel': 'senador', 'David Sandoval': 'senador', 'Ximena Ordenes': 'senador', 'Karim Bianchi': 'senador', 'Alejandro Kusanovic': 'senador', 'Carolina Toha': 'ministro', 'Maya Fernandez': 'ministro', 'Alberto van Klaveren': 'ministro', 'Mario Marcel': 'ministro', 'Alvaro Elizalde': 'ministro', 'Camila Vallejo': 'ministro', 'Nicolas Grau': 'ministro', 'Javiera Toro': 'ministro', 'Nicolas Cataldo': 'ministro', 'Luis Cordero': 'ministro', 'Jeannette Jara': 'ministro', 'Jessica Lopez': 'ministro', 'Ximena Aguilera': 'ministro', 'Carlos Montes': 'ministro', 'Esteban Valenzuela': 'ministro', 'Aurora Williams': 'ministro', 'Juan Carlos Muñoz': 'ministro', 'Marcela Sandoval': 'ministro', 'Diego Pardow': 'ministro', 'Maisa Rojas': 'ministro', 'Jaime Pizarro': 'ministro', 'Antonia Orellana': 'ministro', 'Carolina Arredondo': 'ministro', 'Aisen Etcheverry': 'ministro', 'Gabriel Boric': 'presidente', 'Sebastian Piñera': 'expresidente', 'Michelle Bachelet': 'expresidente', 'Ricardo Lagos Escobar': 'expresidente', 'Eduardo Frei Ruiz-Tagle': 'expresidente', 'Patricio Aylwin': 'expresidente'}
keywords = ["diputado", "diputada", "senador", "senadora", "Cámara", "parlamentarios", "bancada"]

# Inicializa el cliente de BigQuery
client = bigquery.Client()

# Función para descargar datos de BigQuery y preprocesarlos
def preprocess_data():
    # Define tu consulta SQL para extraer datos de la tabla
    query = """
        SELECT title, body, title_1, date FROM `vernal-guide-414621.jgj767.igilñ`
    """

    # Ejecuta la consulta
    query_job = client.query(query)

    # Obtén los resultados de la consulta y conviértelos a un DataFrame de pandas
    df = query_job.to_dataframe()
    
    # Procesa los datos
    processed_data = preprocesar_text_solo_noticias(df)
    
    # Inserta los datos procesados en BigQuery
    insert_to_bigquery(processed_data)

# Función para preprocesar el texto de las noticias
def preprocesar_text_solo_noticias(df):
    # Se añaden entidades detectadas como 'Personas' en una nueva columna, evitando SettingWithCopyWarning
    df.drop_duplicates(subset=['body'], inplace=True)
    df['body_length'] = df['body'].str.len()
    df = df[df['body_length'] > 500]
    df = df[df['body_length'] < 2000]

    def filtrar_noticias(noticia):
        # Búsqueda de palabras clave
        contiene_palabra_clave = any(keyword in noticia.lower() for keyword in keywords)

        # Búsqueda de nombres de personas de la red política
        contiene_nombre_persona = any(re.search(nombre, noticia, re.IGNORECASE) for nombre in personas_red_politica_dict.keys())

        # La noticia se conservará si cumple con al menos uno de los criterios
        return contiene_palabra_clave or contiene_nombre_persona

    df['filtrar'] = df['body'].apply(filtrar_noticias)
    df = df[df['filtrar']]

    return df

# Función para insertar los datos procesados en BigQuery
def insert_to_bigquery(data):
    # Define el ID del proyecto y el ID del dataset donde se encuentra la tabla de destino
    project_id = 'vernal-guide-414621'
    dataset_id = 'news'
    table_id = 'inference'

    # Construye una referencia a la tabla de destino
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Intenta obtener la tabla, si no existe, la creará
    try:
        table = client.get_table(table_ref)
    except NotFound:
        schema = [
            bigquery.SchemaField("body", "STRING"),
            # Agrega aquí más campos si es necesario
        ]
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)

    # Convertir el DataFrame a una lista de diccionarios
    # Convertir el DataFrame a una lista de diccionarios
    rows_to_insert = data.to_dict(orient='records')

    # Convertir valores de fecha a cadena de texto
    for row in rows_to_insert:
        if 'date' in row and isinstance(row['date'], datetime.date):
            row['date'] = row['date'].strftime('%Y-%m-%d')
    # Inserta los datos en la tabla de destino utilizando el esquema existente
    errors = client.insert_rows_json(table_ref, rows_to_insert)

    # Maneja cualquier error que ocurra durante la inserción
    if errors:
        for error in errors:
            print(f'Error al insertar fila: {error}')
    else:
        print('Datos insertados correctamente en BigQuery.')
        
# Esta es la entrada de la Cloud Function
def main(request):
    preprocess_data()
    return "Proceso de preprocesamiento y carga completado."

In [ ]:
preprocess_data()


# Prueba función pulp

### Ejemplo de nueva metodologia (se necesitara agregar 2 columnas, y un conteo de noticias & numero de noticias requeridas a la semana por ahora 750): 

In [8]:
import pulp

# Datos de ejemplo (reemplazar con tus datos reales)
N = 5  # Número de noticias
P = [3, 2, 5, 1, 4]  # Número de personas políticas relevantes mencionadas en cada noticia
L = [1500, 800, 1200, 500, 900]  # Longitud del cuerpo de cada noticia
lambda_ = 0.05  # Parámetro lambda para balancear la función objetivo
k = 3  # Número específico de noticias a seleccionar

# Crear el problema de maximización
prob = pulp.LpProblem("Maximizar_Presencias_Politicas", pulp.LpMaximize)

# Variables de decisión: x_i indica si la noticia i es seleccionada (1) o no (0)
x = [pulp.LpVariable(f"x{i}", cat='Binary') for i in range(N)]

# Función objetivo
prob += pulp.lpSum(P[i] * x[i] for i in range(N)) - lambda_ * pulp.lpSum(L[i] * x[i] for i in range(N))

# Restricción: Seleccionar exactamente k noticias
prob += pulp.lpSum(x) == k, "Seleccionar_k_noticias"

# Resolver el problema
prob.solve()

# Imprimir los resultados
print("Estado de la solución:", pulp.LpStatus[prob.status])
for i in range(N):
    print(f"Noticia {i+1}: {'Seleccionada' if x[i].varValue == 1 else 'No seleccionada'}")

# Valor de la función objetivo
print("Valor de la función objetivo:", pulp.value(prob.objective))



Estado de la solución: Optimal
Noticia 1: No seleccionada
Noticia 2: Seleccionada
Noticia 3: No seleccionada
Noticia 4: Seleccionada
Noticia 5: Seleccionada
Valor de la función objetivo: -103.0


## Generar inferencia con el modelo de HuggingFace y arreglarlas manualmente, posteriormente guardarlas nuevamente en la tabla de BigQuery